In [2]:
import pandas as pd

ratings_movies = pd.read_csv('data/ratings_movies.csv')
ratings_movies.head()

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [5]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [7]:
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


In [8]:
mask = ratings_movies['year_release'] == 1999
ratings_movies[mask].groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [9]:
mask = ratings_movies['year_release'] == 2010
ratings_movies[mask].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [10]:
ratings_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False)


userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [12]:
ratings_movies.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [14]:
mask = ratings_movies['year_release'] == 2018
grouped = ratings_movies[mask].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
grouped[grouped['count']>10].sort_values(
    by='mean',
    ascending=False
)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [15]:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year
pivot = ratings_movies.pivot_table(
    index='year_rating',
    columns='genres',
    values='rating',
    aggfunc='mean'
)
display(pivot)

genres,(no genres listed),Action,Action|Adventure,Action|Adventure|Animation,Action|Adventure|Animation|Children,Action|Adventure|Animation|Children|Comedy,Action|Adventure|Animation|Children|Comedy|Fantasy,Action|Adventure|Animation|Children|Comedy|IMAX,Action|Adventure|Animation|Children|Comedy|Romance,Action|Adventure|Animation|Children|Comedy|Sci-Fi,...,Romance|Thriller,Romance|War,Romance|Western,Sci-Fi,Sci-Fi|IMAX,Sci-Fi|Thriller,Sci-Fi|Thriller|IMAX,Thriller,War,Western
year_rating,,,,,,,,,,,,,,,,,,,,,
1996,NaN,2.730769,3.454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.838095,NaN,3.117647
1997,NaN,3.538462,4.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.400000,NaN,3.923077,NaN,3.000000
1998,NaN,NaN,4.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.800000,NaN,NaN
1999,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000,NaN,NaN,NaN,NaN,4.000000,NaN,3.700000,4.5,4.000000
2000,NaN,2.588235,3.738462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.000,NaN,3.0,3.416667,NaN,2.142857,NaN,3.087912,3.0,4.058824
2001,NaN,3.000000,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,2.500000,NaN,2.500000,NaN,3.477273,3.0,3.111111
2002,NaN,2.750000,4.304348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000,NaN,NaN,3.750000,NaN,3.600000,NaN,3.583333,3.5,3.000000
2003,NaN,3.833333,3.277778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.375,2.5,NaN,2.333333,NaN,3.142857,NaN,3.250000,3.0,4.000000
2004,NaN,2.700000,4.136364,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,...,3.000,3.0,3.5,2.125000,NaN,NaN,NaN,3.464286,3.0,3.800000


In [38]:
orders_df = pd.read_csv('data/orders.csv', sep=';')
orders_df.head()

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5


In [40]:
products_df = pd.read_csv('data/products.csv', sep=';')
products_df.head()


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR


In [41]:
orders_products = pd.concat([orders_df, products_df], axis=1)
display(orders_products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,47.0,Шатны Полосатый рейс,2999.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,51.0,Платье Аленький цветочек,4999.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,53.0,Штаны Цветочная Поляна,4999.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,71.0,Платье Ночная Жизнь,7999.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,74.0,Платье Ночная Жизнь XXXL,8999.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,86.0,"Носки Простые, муж",45.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,91.0,"Носки Честные, муж",50.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,103.0,"Носки Подарочные, муж",199.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,104.0,"Носки Подарочные, жен",249.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,124.0,Носки беговые Camino,999.0,RUR


In [42]:
orders_products[orders_products['Отменен'] == 'Да']['Name']

6    Носки Честные, муж
Name: Name, dtype: object

In [43]:
orders_products['Profit'] = orders_products['Price'] * orders_products['Количество'] 
orders_products[orders_products['Оплачен'] == 'Да'].groupby('ID Покупателя')['Profit'].sum().sort_values(ascending=False)

ID Покупателя
8    3744.0
7     699.0
1       0.0
5       0.0
Name: Profit, dtype: float64